In [3]:
'''

This code is part of the SIPN2 project focused on improving sub-seasonal to seasonal predictions of Arctic Sea Ice. 
If you use this code for a publication or presentation, please cite the reference in the README.md on the
main page (https://github.com/NicWayand/ESIO). 

Questions or comments should be addressed to nicway@uw.edu

Copyright (c) 2018 Nic Wayand

GNU General Public License v3.0


'''

from ecmwfapi import ECMWFDataServer
import os
import sys
import dask
import numpy as np
from esio import download
import datetime

'''
Download models with sea ice forecasts within the 2s2 forecast data archive.

https://software.ecmwf.int/wiki/display/S2S/Models

'''

'\nDownload models with sea ice forecasts within the 2s2 forecast data archive.\n\nhttps://software.ecmwf.int/wiki/display/S2S/Models\n\n'

In [2]:
# Check user defined configuraiton file
if len(sys.argv) < 2:
    raise ValueError('Requires either one arguments [recent] \n or two [start_year, end_yaer] [start_month, end_month] (inclusive) ')

# Get name of configuration file/module
timeperiod = sys.argv[1]
if timeperiod=='recent':
    cd = datetime.datetime.now()
    years = [cd.year]
    months = [cd.month]
else:
    year_range_in = list(map(int, sys.argv[1].strip('[]').split(',')))
    month_range_in = list(map(int, sys.argv[2].strip('[]').split(',')))
    if (len(year_range_in)!=2) | (len(month_range_in)!=2):
        raise ValueError('Year range and month range must be two values (inclusive)')
    years = np.arange(year_range_in[0], year_range_in[1]+1, 1)
    months = np.arange(month_range_in[0], month_range_in[1]+1, 1)
    assert np.all(months>=0), 'months must be >=0'
    assert np.all(months<=12), 'months must be <=12'

ValueError: invalid literal for int() with base 10: '-f'

In [13]:
    
# # Testing
years = [2019]
months = [6]
months = np.arange(1,6,1)
print(years)
print(months)

[2019]
[1 2 3 4 5]


In [4]:
main_dir = '/home/disk/sipn/nicway/data/model'

In [5]:
# Templet dicts for each model for control runs  these are deterministic runs I guess

# Init it
mod_dicts = {}

# ECMWF
mod_dicts['ecmwf'] = {
"class": "s2",
"dataset": "s2s",
"date": "2018-02-01/2018-02-05/2018-02-08/2018-02-12/2018-02-15/2018-02-19/2018-02-22/2018-02-26",
"expver": "prod",
"levtype": "sfc",
"model": "glob",
"origin": "ecmf",
"param": "31",
"step": "0-24/24-48/48-72/72-96/96-120/120-144/144-168/168-192/192-216/216-240/240-264/264-288/288-312/312-336/336-360/360-384/384-408/408-432/432-456/456-480/480-504/504-528/528-552/552-576/576-600/600-624/624-648/648-672/672-696/696-720/720-744/744-768/768-792/792-816/816-840/840-864/864-888/888-912/912-936/936-960/960-984/984-1008/1008-1032/1032-1056/1056-1080/1080-1104",
"stream": "enfo",
"time": "00:00:00",
"type": "cf",
"target": "output",
}

# CMA
mod_dicts['cma'] = {
"class": "s2",
"dataset": "s2s",
"date": "2018-02-01/to/2018-02-28",
"expver": "prod",
"levtype": "sfc",
"model": "glob",
"origin": "babj",
"param": "31",
"step": "0-24/24-48/48-72/72-96/96-120/120-144/144-168/168-192/192-216/216-240/240-264/264-288/288-312/312-336/336-360/360-384/384-408/408-432/432-456/456-480/480-504/504-528/528-552/552-576/576-600/600-624/624-648/648-672/672-696/696-720/720-744/744-768/768-792/792-816/816-840/840-864/864-888/888-912/912-936/936-960/960-984/984-1008/1008-1032/1032-1056/1056-1080/1080-1104/1104-1128/1128-1152/1152-1176/1176-1200/1200-1224/1224-1248/1248-1272/1272-1296/1296-1320/1320-1344/1344-1368/1368-1392/1392-1416/1416-1440",
"stream": "enfo",
"time": "00:00:00",
"type": "cf",
"target": "output",
}


# Metreo France
mod_dicts['metreofr'] = {
"class": "s2",
"dataset": "s2s",
"date": "2018-02-01/to/2018-02-22/by/7",
"expver": "prod",
"levtype": "sfc",
"model": "glob",
"origin": "lfpw",
"param": "31",
"step": "0-24/24-48/48-72/72-96/96-120/120-144/144-168/168-192/192-216/216-240/240-264/264-288/288-312/312-336/336-360/360-384/384-408/408-432/432-456/456-480/480-504/504-528/528-552/552-576/576-600/600-624/624-648/648-672/672-696/696-720/720-744/744-768",
"stream": "enfo",
"time": "00:00:00",
"type": "cf",
"target": "output",
}

# NCEP
mod_dicts['ncep'] = {
"class": "s2",
"dataset": "s2s",
"date": "2018-02-01/to/2018-02-28",
"expver": "prod",
"levtype": "sfc",
"model": "glob",
"origin": "kwbc",
"param": "31",
"step": "24-48/48-72/72-96/96-120/120-144/144-168/168-192/192-216/216-240/240-264/264-288/288-312/312-336/336-360/360-384/384-408/408-432/432-456/456-480/480-504/504-528/528-552/552-576/576-600/600-624/624-648/648-672/672-696/696-720/720-744/744-768/768-792/792-816/816-840/840-864/864-888/888-912/912-936/936-960/960-984/984-1008/1008-1032/1032-1056",
"stream": "enfo",
"time": "00:00:00",
"type": "cf",
"target": "output",
}

# UKMO
mod_dicts['ukmo'] = {
"class": "s2",
"dataset": "s2s",
"date": "2018-02-01/to/2018-02-28",
"expver": "prod",
"levtype": "sfc",
"model": "glob",
"origin": "egrr",
"param": "31",
"step": "0-24/24-48/48-72/72-96/96-120/120-144/144-168/168-192/192-216/216-240/240-264/264-288/288-312/312-336/336-360/360-384/384-408/408-432/432-456/456-480/480-504/504-528/528-552/552-576/576-600/600-624/624-648/648-672/672-696/696-720/720-744/744-768/768-792/792-816/816-840/840-864/864-888/888-912/912-936/936-960/960-984/984-1008/1008-1032/1032-1056/1056-1080/1080-1104/1104-1128/1128-1152/1152-1176/1176-1200/1200-1224/1224-1248/1248-1272/1272-1296/1296-1320/1320-1344/1344-1368/1368-1392/1392-1416/1416-1440",
"stream": "enfo",
"time": "00:00:00",
"type": "cf",
"target": "output",
}


# KMA
mod_dicts['kma'] = {
"class": "s2",
"dataset": "s2s",
"date": "2018-02-01/to/2018-02-28",
"expver": "prod",
"levtype": "sfc",
"model": "glob",
"origin": "rksl",
"param": "31",
"step": "0-24/24-48/48-72/72-96/96-120/120-144/144-168/168-192/192-216/216-240/240-264/264-288/288-312/312-336/336-360/360-384/384-408/408-432/432-456/456-480/480-504/504-528/528-552/552-576/576-600/600-624/624-648/648-672/672-696/696-720/720-744/744-768/768-792/792-816/816-840/840-864/864-888/888-912/912-936/936-960/960-984/984-1008/1008-1032/1032-1056/1056-1080/1080-1104/1104-1128/1128-1152/1152-1176/1176-1200/1200-1224/1224-1248/1248-1272/1272-1296/1296-1320/1320-1344/1344-1368/1368-1392/1392-1416/1416-1440",
"stream": "enfo",
"time": "00:00:00",
"type": "cf",
"target": "output",
}


#mod_dicts.keys()

In [11]:
# Init it  these are the perturbed runs which have an ensemble
pf_mod_dicts = {}

# ECMWF
pf_mod_dicts['ecmwf'] = {
"class": "s2",
"dataset": "s2s",
"date": "2018-02-01/2018-02-05/2018-02-08/2018-02-12/2018-02-15/2018-02-19/2018-02-22/2018-02-26",
"expver": "prod",
"levtype": "sfc",
"model": "glob",
"origin": "ecmf",
"param": "31",
"step": "0-24/24-48/48-72/72-96/96-120/120-144/144-168/168-192/192-216/216-240/240-264/264-288/288-312/312-336/336-360/360-384/384-408/408-432/432-456/456-480/480-504/504-528/528-552/552-576/576-600/600-624/624-648/648-672/672-696/696-720/720-744/744-768/768-792/792-816/816-840/840-864/864-888/888-912/912-936/936-960/960-984/984-1008/1008-1032/1032-1056/1056-1080/1080-1104",
"stream": "enfo",
"time": "00:00:00",
"type": "pf",
"number": "1/2/3/4/5/6/7/8/9/10/11/12/13/14/15/16/17/18/19/20/21/22/23/24/25/26/27/28/29/30/31/32/33/34/35/36/37/38/39/40/41/42/43/44/45/46/47/48/49/50",
"target": "output",
}

# CMA
pf_mod_dicts['cma'] = {
"class": "s2",
"dataset": "s2s",
"date": "2018-02-01/to/2018-02-28",
"expver": "prod",
"levtype": "sfc",
"model": "glob",
"origin": "babj",
"param": "31",
"step": "0-24/24-48/48-72/72-96/96-120/120-144/144-168/168-192/192-216/216-240/240-264/264-288/288-312/312-336/336-360/360-384/384-408/408-432/432-456/456-480/480-504/504-528/528-552/552-576/576-600/600-624/624-648/648-672/672-696/696-720/720-744/744-768/768-792/792-816/816-840/840-864/864-888/888-912/912-936/936-960/960-984/984-1008/1008-1032/1032-1056/1056-1080/1080-1104/1104-1128/1128-1152/1152-1176/1176-1200/1200-1224/1224-1248/1248-1272/1272-1296/1296-1320/1320-1344/1344-1368/1368-1392/1392-1416/1416-1440",
"stream": "enfo",
"time": "00:00:00",
"type": "pf",
"target": "output",
}


# Metreo France
pf_mod_dicts['metreofr'] = {
"class": "s2",
"dataset": "s2s",
"date": "2018-02-01/to/2018-02-22/by/7",
"expver": "prod",
"levtype": "sfc",
"model": "glob",
"origin": "lfpw",
"param": "31",
"step": "0-24/24-48/48-72/72-96/96-120/120-144/144-168/168-192/192-216/216-240/240-264/264-288/288-312/312-336/336-360/360-384/384-408/408-432/432-456/456-480/480-504/504-528/528-552/552-576/576-600/600-624/624-648/648-672/672-696/696-720/720-744/744-768",
"stream": "enfo",
"time": "00:00:00",
"type": "pf",
"number": "1/2/3/4/5/6/7/8/9/10/11/12/13/14/15/16/17/18/19/20/21/22/23/24/25/26/27/28/29/30/31/32/33/34/35/36/37/38/39/40/41/42/43/44/45/46/47/48/49/50",
"target": "output",
}

# NCEP
pf_mod_dicts['ncep'] = {
"class": "s2",
"dataset": "s2s",
"date": "2018-02-01/to/2018-02-28",
"expver": "prod",
"levtype": "sfc",
"model": "glob",
"origin": "kwbc",
"param": "31",
"step": "24-48/48-72/72-96/96-120/120-144/144-168/168-192/192-216/216-240/240-264/264-288/288-312/312-336/336-360/360-384/384-408/408-432/432-456/456-480/480-504/504-528/528-552/552-576/576-600/600-624/624-648/648-672/672-696/696-720/720-744/744-768/768-792/792-816/816-840/840-864/864-888/888-912/912-936/936-960/960-984/984-1008/1008-1032/1032-1056",
"stream": "enfo",
"time": "00:00:00",
"type": "pf",
"number": "1/2/3/4/5/6/7/8/9/10/11/12/13/14/15",
"target": "output",
}

# UKMO
pf_mod_dicts['ukmo'] = {
"class": "s2",
"dataset": "s2s",
"date": "2018-02-01/to/2018-02-28",
"expver": "prod",
"levtype": "sfc",
"model": "glob",
"origin": "egrr",
"param": "31",
"step": "0-24/24-48/48-72/72-96/96-120/120-144/144-168/168-192/192-216/216-240/240-264/264-288/288-312/312-336/336-360/360-384/384-408/408-432/432-456/456-480/480-504/504-528/528-552/552-576/576-600/600-624/624-648/648-672/672-696/696-720/720-744/744-768/768-792/792-816/816-840/840-864/864-888/888-912/912-936/936-960/960-984/984-1008/1008-1032/1032-1056/1056-1080/1080-1104/1104-1128/1128-1152/1152-1176/1176-1200/1200-1224/1224-1248/1248-1272/1272-1296/1296-1320/1320-1344/1344-1368/1368-1392/1392-1416/1416-1440",
"stream": "enfo",
"time": "00:00:00",
"type": "pf",
"number": "1/2/3",
"target": "output",
}


# KMA
pf_mod_dicts['kma'] = {
"class": "s2",
"dataset": "s2s",
"date": "2018-02-01/to/2018-02-28",
"expver": "prod",
"levtype": "sfc",
"model": "glob",
"origin": "rksl",
"param": "31",
"step": "0-24/24-48/48-72/72-96/96-120/120-144/144-168/168-192/192-216/216-240/240-264/264-288/288-312/312-336/336-360/360-384/384-408/408-432/432-456/456-480/480-504/504-528/528-552/552-576/576-600/600-624/624-648/648-672/672-696/696-720/720-744/744-768/768-792/792-816/816-840/840-864/864-888/888-912/912-936/936-960/960-984/984-1008/1008-1032/1032-1056/1056-1080/1080-1104/1104-1128/1128-1152/1152-1176/1176-1200/1200-1224/1224-1248/1248-1272/1272-1296/1296-1320/1320-1344/1344-1368/1368-1392/1392-1416/1416-1440",
"stream": "enfo",
"time": "00:00:00",
"type": "pf",
"number": "1/2/3",
"target": "output",
}


In [7]:
Include_NoSeaIceModels = False
if Include_NoSeaIceModels:

    # bom
    mod_dicts['bom'] = {
    "class": "s2",
    "dataset": "s2s",
    "date": "2018-02-01/2018-02-04/2018-02-08/2018-02-11/2018-02-15/2018-02-18/2018-02-22/2018-02-25",
    "expver": "prod",
    "levtype": "sfc",
    "model": "glob",
    "origin": "ammc",
    "param": "31",
    "step": "0-24/24-48/48-72/72-96/96-120/120-144/144-168/168-192/192-216/216-240/240-264/264-288/288-312/312-336/336-360/360-384/384-408/408-432/432-456/456-480/480-504/504-528/528-552/552-576/576-600/600-624/624-648/648-672/672-696/696-720/720-744/744-768/768-792/792-816/816-840/840-864/864-888/888-912/912-936/936-960/960-984/984-1008/1008-1032/1032-1056/1056-1080/1080-1104/1104-1128/1128-1152/1152-1176/1176-1200/1200-1224/1224-1248/1248-1272/1272-1296/1296-1320/1320-1344/1344-1368/1368-1392/1392-1416/1416-1440/1440-1464/1464-1488",
    "stream": "enfo",
    "time": "00:00:00",
    "type": "cf",
    "target": "output",
    }

    # HCMR
    mod_dicts['hcmr'] = {
    "class": "s2",
    "dataset": "s2s",
    "date": "2018-02-01/to/2018-02-22/by/7",
    "expver": "prod",
    "levtype": "sfc",
    "model": "glob",
    "origin": "rums",
    "param": "31",
    "step": "0-24/24-48/48-72/72-96/96-120/120-144/144-168/168-192/192-216/216-240/240-264/264-288/288-312/312-336/336-360/360-384/384-408/408-432/432-456/456-480/480-504/504-528/528-552/552-576/576-600/600-624/624-648/648-672/672-696/696-720/720-744/744-768/768-792/792-816/816-840/840-864/864-888/888-912/912-936/936-960/960-984/984-1008/1008-1032/1032-1056/1056-1080/1080-1104/1104-1128/1128-1152/1152-1176/1176-1200/1200-1224/1224-1248/1248-1272/1272-1296/1296-1320/1320-1344/1344-1368/1368-1392/1392-1416/1416-1440/1440-1464",
    "stream": "enfo",
    "time": "00:00:00",
    "type": "cf",
    "target": "output",
    }

    # ISAC-CNR
    mod_dicts['isaccnr'] = {
    "class": "s2",
    "dataset": "s2s",
    "date": "2018-02-01/to/2018-02-22/by/7",
    "expver": "prod",
    "levtype": "sfc",
    "model": "glob",
    "origin": "isac",
    "param": "31",
    "step": "0-24/24-48/48-72/72-96/96-120/120-144/144-168/168-192/192-216/216-240/240-264/264-288/288-312/312-336/336-360/360-384/384-408/408-432/432-456/456-480/480-504/504-528/528-552/552-576/576-600/600-624/624-648/648-672/672-696/696-720/720-744/744-768",
    "stream": "enfo",
    "time": "00:00:00",
    "type": "cf",
    "target": "output",
    }


    # JMA
    mod_dicts['jma'] = {
    "class": "s2",
    "dataset": "s2s",
    "date": "2018-02-07/to/2018-02-28/by/7",
    "expver": "prod",
    "levtype": "sfc",
    "model": "glob",
    "origin": "rjtd",
    "param": "31",
    "step": "12-36/36-60/60-84/84-108/108-132/132-156/156-180/180-204/204-228/228-252/252-276/276-300/300-324/324-348/348-372/372-396/396-420/420-444/444-468/468-492/492-516/516-540/540-564/564-588/588-612/612-636/636-660/660-684/684-708/708-732/732-756/756-780",
    "stream": "enfo",
    "time": "12:00:00",
    "type": "cf",
    "target": "output",
    }

    # ECCC
    mod_dicts['eccc'] = {
    "class": "s2",
    "dataset": "s2s",
    "date": "2018-02-01/to/2018-02-22/by/7",
    "expver": "prod",
    "levtype": "sfc",
    "model": "glob",
    "origin": "cwao",
    "param": "31",
    "step": "0-24/24-48/48-72/72-96/96-120/120-144/144-168/168-192/192-216/216-240/240-264/264-288/288-312/312-336/336-360/360-384/384-408/408-432/432-456/456-480/480-504/504-528/528-552/552-576/576-600/600-624/624-648/648-672/672-696/696-720/720-744/744-768",
    "stream": "enfo",
    "time": "00:00:00",
    "type": "cf",
    "target": "output",
    }

In [14]:
# Download by month, wait for each month to finish
for cy in years:
    print(cy)
    for cm in months:
        print(cm)
        download.download_data_by_month(dataclass='s2s', main_dir=main_dir, mod_dicts=pf_mod_dicts, cy=cy, cm=cm)

2019
1
2019-01-01/to/2019-01-31
ecmwf
cma
metreofr
ncep
ukmo
kma
2019-07-05 18:25:44 ECMWF API python library 1.5.4
2019-07-05 18:25:44 ECMWF API at https://api.ecmwf.int/v1
2019-07-05 18:25:44 ECMWF API python library 1.5.4
2019-07-05 18:25:44 ECMWF API at https://api.ecmwf.int/v1
2019-07-05 18:25:44 ECMWF API python library 1.5.42019-07-05 18:25:44 ECMWF API python library 1.5.4

2019-07-05 18:25:44 ECMWF API at https://api.ecmwf.int/v1
2019-07-05 18:25:44 ECMWF API at https://api.ecmwf.int/v1
2019-07-05 18:25:44 ECMWF API python library 1.5.4
2019-07-05 18:25:44 ECMWF API at https://api.ecmwf.int/v1
2019-07-05 18:25:44 ECMWF API python library 1.5.4
2019-07-05 18:25:44 ECMWF API at https://api.ecmwf.int/v1
2019-07-05 18:25:45 Welcome Cecilia Bitz
2019-07-05 18:25:45 Welcome Cecilia Bitz
2019-07-05 18:25:45 Welcome Cecilia Bitz
2019-07-05 18:25:45 Welcome Cecilia Bitz
2019-07-05 18:25:45 Welcome Cecilia Bitz
2019-07-05 18:25:45 Welcome Cecilia Bitz
2019-07-05 18:25:46 In case of prob

Calling 'nice mars /tmp/68/92/tmp-_marstJGj1a.req'
mars - WARN -
mars - WARN - From 29 January 2019 10AM (UTC) MARS uses the interpolation
mars - WARN - provided by the MIR library. For more details, see
mars - WARN - https://confluence.ecmwf.int/display/UDOC/MARS+interpolation+with+MIR
mars - WARN -
PPDIR is /var/tmp/ppdir/x86_64
mars - INFO   - 20190706.012549 - Welcome to MARS
mars - INFO   - 20190706.012549 - MARS Client build stamp: 20190607111047
mars - INFO   - 20190706.012549 - MARS Client version: 6.25.3
mars - INFO   - 20190706.012549 - MIR version: 1.2.5
mars - INFO   - 20190706.012549 - Using ecCodes version 2.12.6
mars - INFO   - 20190706.012549 - Using odb_api version: 0.15.10 (file format version: 0.5)
mars - INFO   - 20190706.012549 - Maximum retrieval size is 30.00 G
retrieve,origin=kwbc,stream=enfo,levtype=sfc,number=1/2/3/4/5/6/7/8/9/10/11/12/13/14/15,expver=prod,padding=0,step=24-48/48-72/72-96/96-120/120-144/144-168/168-192/192-216/216-240/240-264/264-288/288-312/3

Calling 'nice mars /tmp/3a/69/tmp-_marsYqGjqC.req'
mars - WARN -
mars - WARN - From 29 January 2019 10AM (UTC) MARS uses the interpolation
mars - WARN - provided by the MIR library. For more details, see
mars - WARN - https://confluence.ecmwf.int/display/UDOC/MARS+interpolation+with+MIR
mars - WARN -
PPDIR is /var/tmp/ppdir/x86_64
mars - INFO   - 20190706.012654 - Welcome to MARS
mars - INFO   - 20190706.012654 - MARS Client build stamp: 20190607111047
mars - INFO   - 20190706.012654 - MARS Client version: 6.25.3
mars - INFO   - 20190706.012654 - MIR version: 1.2.5
mars - INFO   - 20190706.012654 - Using ecCodes version 2.12.6
mars - INFO   - 20190706.012654 - Using odb_api version: 0.15.10 (file format version: 0.5)
mars - INFO   - 20190706.012654 - Maximum retrieval size is 30.00 G
retrieve,origin=ecmf,stream=enfo,levtype=sfc,number=1/2/3/4/5/6/7/8/9/10/11/12/13/14/15/16/17/18/19/20/21/22/23/24/25/26/27/28/29/30/31/32/33/34/35/36/37/38/39/40/41/42/43/44/45/46/47/48/49/50,expver=prod,

mars - INFO   - 20190706.012745 - 5580 fields retrieved from 'marsth'
mars - WARN   - 20190706.012745 - Visiting database marsth : expected 0, got 5580
mars - INFO   - 20190706.012745 - Request time:  wall: 27 sec  cpu: 24 sec
mars - INFO   - 20190706.012745 -   Read from network: 335.01 Mbyte(s) in < 1 sec [637.47 Mbyte/sec]
mars - INFO   - 20190706.012745 -   Processing in marsth: wall: 2 sec
mars - INFO   - 20190706.012745 -   Visiting marsth: wall: 27 sec
mars - INFO   - 20190706.012745 -   Post-processing: wall: 15 sec cpu: 15 sec
mars - INFO   - 20190706.012745 -   Writing to target file: 335.01 Mbyte(s) in < 1 sec [373.74 Mbyte/sec]
mars - INFO   - 20190706.012745 - Memory used: 35.54 Mbyte(s)
mars - INFO   - 20190706.012745 - No errors reported
Process '['nice', 'mars', '/tmp/4e/b9/tmp-_marsIUWr6o.req']' finished
2019-07-05 18:27:47 Request is complete
2019-07-05 18:27:47 Transfering 335.01 Mbytes into /home/disk/sipn/nicway/data/model/ukmo/forecast/native/ukmo_2019_01.grib
201

mars - INFO   - 20190706.013318 - Transfering 513084600 bytes
mars - INFO   - 20190706.013340 - 6400 fields retrieved from 'marsth'
mars - WARN   - 20190706.013340 - Visiting database marsth : expected 0, got 6400
mars - INFO   - 20190706.013340 - Request time:  wall: 25 sec  cpu: 24 sec
mars - INFO   - 20190706.013340 -   Read from network: 378.31 Mbyte(s) in < 1 sec [970.92 Mbyte/sec]
mars - INFO   - 20190706.013340 -   Visiting marsth: wall: 25 sec
mars - INFO   - 20190706.013340 -   Post-processing: wall: 15 sec cpu: 15 sec
mars - INFO   - 20190706.013340 -   Writing to target file: 378.31 Mbyte(s) in < 1 sec [601.99 Mbyte/sec]
mars - INFO   - 20190706.013340 - Memory used: 35.54 Mbyte(s)
mars - INFO   - 20190706.013340 - No errors reported
Process '['nice', 'mars', '/tmp/32/d2/tmp-_marsXLeBC9.req']' finished
2019-07-05 18:33:42 Request is complete
2019-07-05 18:33:42 Transfering 378.308 Mbytes into /home/disk/sipn/nicway/data/model/metreofr/forecast/native/metreofr_2019_02.grib
20

...

mars - INFO   - 20190706.014020 - 19995 fields retrieved from 'marsth'
mars - WARN   - 20190706.014020 - Visiting database marsth : expected 0, got 19995
mars - INFO   - 20190706.014020 - Request time:  wall: 1 min 21 sec  cpu: 1 min 19 sec
mars - INFO   - 20190706.014020 -   Read from network: 541.74 Mbyte(s) in < 1 sec [687.08 Mbyte/sec]
mars - INFO   - 20190706.014020 -   Processing in marsth: wall: 2 sec
mars - INFO   - 20190706.014020 -   Visiting marsth: wall: 1 min 21 sec
mars - INFO   - 20190706.014020 -   Post-processing: wall: 50 sec cpu: 49 sec
mars - INFO   - 20190706.014020 -   Writing to target file: 541.74 Mbyte(s) in 1 sec  [457.06 Mbyte/sec] cpu: 1 sec
mars - INFO   - 20190706.014020 - Memory used: 35.64 Mbyte(s)
mars - INFO   - 20190706.014020 - No errors reported
Process '['nice', 'mars', '/tmp/b3/58/tmp-_marsUiWACZ.req']' finished
2019-07-05 18:40:28 Request is complete
2019-07-05 18:40:28 Transfering 541.742 Mbytes into /home/disk/sipn/nicway/data/model/ncep/foreca

2019-07-05 18:46:57 Request is active
Calling 'nice mars /tmp/a4/df/tmp-_marsyHvzyZ.req'
mars - WARN -
mars - WARN - From 29 January 2019 10AM (UTC) MARS uses the interpolation
mars - WARN - provided by the MIR library. For more details, see
mars - WARN - https://confluence.ecmwf.int/display/UDOC/MARS+interpolation+with+MIR
mars - WARN -
PPDIR is /var/tmp/ppdir/x86_64
mars - INFO   - 20190706.014657 - Welcome to MARS
mars - INFO   - 20190706.014657 - MARS Client build stamp: 20190607111047
mars - INFO   - 20190706.014657 - MARS Client version: 6.25.3
mars - INFO   - 20190706.014657 - MIR version: 1.2.5
mars - INFO   - 20190706.014657 - Using ecCodes version 2.12.6
mars - INFO   - 20190706.014657 - Using odb_api version: 0.15.10 (file format version: 0.5)
mars - INFO   - 20190706.014657 - Maximum retrieval size is 30.00 G
retrieve,origin=ecmf,stream=enfo,levtype=sfc,number=1/2/3/4/5/6/7/8/9/10/11/12/13/14/15/16/17/18/19/20/21/22/23/24/25/26/27/28/29/30/31/32/33/34/35/36/37/38/39/40/41/4

2019-07-05 18:53:01 Request submitted
2019-07-05 18:53:01 Request id: 5d1ffefda2de19b422185c1b
2019-07-05 18:53:01 Request is submitted
2019-07-05 18:53:01 Request submitted2019-07-05 18:53:01 Request submitted
2019-07-05 18:53:01 Request id: 5d1ffefdffde1cfc1195961a
2019-07-05 18:53:01 Request id: 5d1ffefd30dc1993e4161e92
2019-07-05 18:53:01 Request is submitted

2019-07-05 18:53:01 Request is submitted
2019-07-05 18:53:01 Request submitted
2019-07-05 18:53:01 Request id: 5d1ffefd87e11c310e468e54
2019-07-05 18:53:01 Request is submitted
2019-07-05 18:53:01 Request submitted
2019-07-05 18:53:01 Request id: 5d1ffefd13e01ccb22d9a956
2019-07-05 18:53:01 Request is submitted
2019-07-05 18:53:01 Request submitted
2019-07-05 18:53:01 Request id: 5d1ffefd72ce19a3f72d4424
2019-07-05 18:53:01 Request is submitted
2019-07-05 18:53:02 Request is queued
Calling 'nice mars /tmp/f1/72/tmp-_marsvRv9Gu.req'
mars - WARN -
mars - WARN - From 29 January 2019 10AM (UTC) MARS uses the interpolation
mars - 

mars - INFO   - 20190706.015308 - Server task is 545 [marsth]
mars - INFO   - 20190706.015308 - Request cost: 5,580 fields, 335.01 Mbytes online, nodes: mvr01 [marsth]
mars - INFO   - 20190706.015309 - Transfering 351283320 bytes
mars - INFO   - 20190706.015330 - 5580 fields retrieved from 'marsth'
mars - WARN   - 20190706.015330 - Visiting database marsth : expected 0, got 5580
mars - INFO   - 20190706.015330 - Request time:  wall: 23 sec  cpu: 20 sec
mars - INFO   - 20190706.015330 -   Read from network: 298.42 Mbyte(s) in < 1 sec [583.38 Mbyte/sec]
mars - INFO   - 20190706.015330 -   Processing in marsth: wall: 2 sec
mars - INFO   - 20190706.015330 -   Visiting marsth: wall: 23 sec
mars - INFO   - 20190706.015330 -   Post-processing: wall: 13 sec cpu: 13 sec
mars - INFO   - 20190706.015330 -   Writing to target file: 298.42 Mbyte(s) in < 1 sec [566.49 Mbyte/sec]
mars - INFO   - 20190706.015330 - Memory used: 35.51 Mbyte(s)
mars - INFO   - 20190706.015330 - No errors reported
Process

Calling 'nice mars /tmp/79/03/tmp-_marszBA67l.req'
mars - WARN -
mars - WARN - From 29 January 2019 10AM (UTC) MARS uses the interpolation
mars - WARN - provided by the MIR library. For more details, see
mars - WARN - https://confluence.ecmwf.int/display/UDOC/MARS+interpolation+with+MIR
mars - WARN -
PPDIR is /var/tmp/ppdir/x86_64
mars - INFO   - 20190706.015442 - Welcome to MARS
mars - INFO   - 20190706.015442 - MARS Client build stamp: 20190607111047
mars - INFO   - 20190706.015442 - MARS Client version: 6.25.3
mars - INFO   - 20190706.015442 - MIR version: 1.2.5
mars - INFO   - 20190706.015442 - Using ecCodes version 2.12.6
mars - INFO   - 20190706.015442 - Using odb_api version: 0.15.10 (file format version: 0.5)
mars - INFO   - 20190706.015442 - Maximum retrieval size is 30.00 G
retrieve,origin=lfpw,stream=enfo,levtype=sfc,number=1/2/3/4/5/6/7/8/9/10/11/12/13/14/15/16/17/18/19/20/21/22/23/24/25/26/27/28/29/30/31/32/33/34/35/36/37/38/39/40/41/42/43/44/45/46/47/48/49/50,expver=prod,